# ■ 日本腎・血液浄化AI学会主催「よくわかる寺子屋セミナー」

# 第5回「深層学習による画像問題」　演習プログラム 【重要なCommandのみ】

## □ 文責　学術委員会副委員長　岩藤和広

# 0. 環境設定

- MyDriveの中にTelakoya というフォルダーを作成し、その中に tela5 というフォルダーを作成する
- フォルダー tela5 の中に model というフォルダーを作る。

- 最初に、このnotebookをGoogle Driveにmount(Google driveに接続)する。
- その方法は、https://github.com/kiwindow/telakoya の中の「Google Colabortoryの使い方」を参照。
- 【重要】 mountする前に
- 「ランタイム」➡「ランタイムのタイプを変更」➡「ハードウェアアクセラレータ」を「GPU」に切り替えて保存する。
- 　これを行わないと、エラーが出たり、計算に長い時間がかかる。
- mount出来ると、左のサイドバーに drive というフォルダーが現れる。
- もし drive というフォルダーがなければ、次のセルを実行する。

In [ ]:
# Gogle driveにmountする
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 以下のコマンドでどの種類のGPUに接続したかが分かる
!nvidia-smi

# 1. ライブラリやデータの準備

## 1) ライブラリの読み込み

- Tensorflow2のwrapperであるKerasを利用する。
- 深層学習はTensorflow2かPyTorchというframeworkを利用するが、どちらもプログラムがやや煩雑。
- KerasはTensorflow2を機械学習のsklearnのような書式で書けるようにしたもの。
- Tensorflow2は企業などで、PyTorchは研究機関などで、それぞれ利用されることが多い。

In [ ]:
# ライブラリの読み込み
import tensorflow as tf
from tensorflow import keras

import numpy as np  # 数値計算のライブラリ
import matplotlib.pyplot as plt  # 図示のライブラリ
%matplotlib inline

## 2) データの読み込み

- mnist と呼ばれる手書き数字(0〜9)の画像データを利用する。
- mnist には、学習データが60,000字、検証データが10,000字ある。
- データは、28×28個の整数(0〜255)からなる行列で形成されている。
- 一つ一つの文字には、0〜9の正解データも付いている。

In [ ]:
# データの読み込み
from keras.datasets import mnist
(x_train, t_train), (x_test, t_test) = mnist.load_data()

## 3) 画像データの正規化・浮動小数点化とChannelの次元の付加

- 整数のデータを、0〜1の浮動小数点(float32)に正規化する。
- 個々のデータの形を、(縦の数、横の数、Channel)にする。

In [ ]:
# 画像データの正規化(0〜1)と小数化(float32)
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html

In [ ]:
# 個々のデータの形を 28 x 28 x 1 とする (channelの次元を最後に加える)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

## 4) 正解データをOne Hot vectorにする

https://keras.io/ja/utils/#to_categorical

- モデルの損失関数が Categorical Crossentropy で、分類結果毎の確率が出る場合、正解データを One Hot vectorにする必要がある。

In [ ]:
from keras.utils import to_categorical
t_train = to_categorical(t_train, 10)
t_test = to_categorical(t_test, 10)

# 2. 三層のPerceptronによる画像分類

## 1) モデルの設計

https://www.tensorflow.org/guide/keras/sequential_model?hl=ja

In [ ]:
# Neural Networkを形成するモジュールの読み込み
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout

In [ ]:
# 三層のPerceptronの作製。中間層の活性化関数はRelu、出力層の活性化関数はsoftmaxとする。
model = Sequential([
    Flatten(input_shape=(28,28,1)),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

In [ ]:
# モデルの構成の確認
model.summary()

## 2) モデルの学習

https://www.tensorflow.org/guide/keras/train_and_evaluate?hl=ja

- 損失関数が Categorical Crossentropyの場合、正解データはOne Hot vectorにする必要がある。

In [ ]:
# 人間が読み書きする source code を、機会が読み書きする machine code にコンパイルする。
# その際、1) 最適化アルゴリズム、2) 損失関数、3) 評価指標なども設定する。
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# モデルの学習を実行する。trainとかfitと呼ばれるmethodで実行する。
# その際、1) training data : validation dataの比　2) バッチサイズ 3) エポック数も設定する。
history = model.fit(x_train, t_train, validation_split = 0.2, batch_size = 128, epochs = 10)

In [ ]:
# 学習過程の図示

# Access the training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Plot of the training and validation loss
epochs = np.arange(1, len(train_loss) + 1)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot　of the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## 3) モデルの検証

https://www.tensorflow.org/guide/keras/train_and_evaluate?hl=ja

In [ ]:
# 予測モデルの精度の検証
score = model.evaluate(x_test, t_test, verbose = 0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

## 4) モデルの保存と復元

https://www.tensorflow.org/guide/keras/save_and_serialize?hl=ja

In [ ]:
# モデルの保存。HDF5ファイルで保存される。
model.save('/content/drive/MyDrive/Telakoya/tela5/model/perceptron.h5')

In [ ]:
# モデルの復元。
from keras.models import load_model
model2 = load_model('/content/drive/MyDrive/Telakoya/tela5/model/perceptron.h5')

In [ ]:
# 保存前のモデルによる予測結果
pred = model.predict(x_test[0:1])
pred

In [ ]:
# 復元したモデルによる予測結果
pred2 = model2.predict(x_test[0:1])
pred2

## 補題 Fashion mnistによる画像分類

In [ ]:
# Googe driveへのマウント。既に行われている場合は評価は不要。
from google.colab import drive
drive.mount('/content/drive')

https://www.tensorflow.org/tutorials/keras/classification?hl=ja

- Fashion mnistというデータセットがあり、mnistと同じ構造で作られている。
- これまでおこなってきたPerceptronによるモデルで、Fashion mnistも分類できる。
- 以下のように、読み込むデータを入れ替えるだけでよい


In [ ]:
# Fashion mnistの読み込み
import tensorflow
from tensorflow import keras
from keras.datasets import fashion_mnist
(x_train, t_train), (x_test, t_test) = fashion_mnist.load_data()

In [ ]:
# Fashion mnistのデータの前処理

# データの正規化 (0〜1)
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

# データの形を 28 x 28 x 1 にする (channelを加える)
import numpy as np
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# 正解データをone hot vectorにする
t_train = keras.utils.to_categorical(t_train, 10)
t_test = keras.utils.to_categorical(t_test, 10)

# 10種類の画像
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
# 最初の10個の画像の図示
import matplotlib.pyplot as plt
%matplotlib inline
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.title('Label: ' + str(i))
    plt.imshow(x_train[i])

In [ ]:
# Neural Networkを形成するモジュールの読み込み
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout

# 三層のPerceptronの作製。中間層の活性化関数はRelu、出力層の活性化関数はsoftmaxとする。
model = Sequential([
    Flatten(input_shape=(28,28,1)),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

# モデルの構成の確認
model.summary()

In [ ]:
# 人間が読み書きする source code を、機会が読み書きする machine code にコンパイルする。
# その際、1) 最適化アルゴリズム、2) 損失関数、3) 評価指標なども設定する。
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# モデルの学習を実行する。trainとかfitと呼ばれるmethodで実行する。
# その際、1) training data : validation dataの比　2) バッチサイズ 3) エポック数も設定する。
history = model.fit(x_train, t_train, validation_split = 0.2, batch_size = 128, epochs = 10)

# 学習過程の図示

# Access the training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Plot of the training and validation loss
epochs = np.arange(1, len(train_loss) + 1)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot　of the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# 検証データでモデルの予測精度を検証する
score = model.evaluate(x_test, t_test, verbose = 0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

In [ ]:
# モデルの保存。HDF5ファイルで保存される。
model.save('/content/drive/MyDrive/Telakoya/tela5/model/perceptronFM.h5')

In [ ]:
# モデルの復元。
from keras.models import load_model
model2 = load_model('/content/drive/MyDrive/Telakoya/tela5/model/perceptronFM.h5')

In [ ]:
# 保存前のモデルによる予測結果
pred = model.predict(x_test[0:1])
pred

In [ ]:
# 復元したモデルによる予測結果
pred2 = model2.predict(x_test[0:1])
pred2

# 3. CNNによる画像分類

https://www.tensorflow.org/tutorials/images/cnn?hl=ja

In [ ]:
# Googe driveへのマウント。既に行われている場合は評価は不要。
from google.colab import drive
drive.mount('/content/drive')

## 1) ライブラリの読み込み

- Tensorflow2のwrapperであるKerasを利用する。


In [ ]:
# ライブラリの読み込み
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import numpy as np  # 数値計算のライブラリ
import matplotlib.pyplot as plt  # 図示のライブラリ
%matplotlib inline

## 2) データの読み込み

- CIFAR10 と呼ばれるカラー画像データを利用する。
- CIFAR10 には、学習データが60,000字、検証データが10,000字ある。
- データは、32×32個の整数(0〜255)が3組ある行列(テンソル)で形成されている。
- 一つ一つの文字には、0〜9の正解データも付いている。

In [ ]:
# データの読み込み
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

- モデルの損失関数がSparse Categorical Crossentropyの場合、正解データは一次元でよい。

In [ ]:
# 最初の25個の画像の図示
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

## 3) モデルの設計

In [ ]:
# CNN (Convolutional Neural Network)の構造。最初に空のSequentialをインスタンス化し、それにLayerを加えてゆく形を取っている。
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

# モデルのアーキテクチャ(構造)の表示
model.summary()

## 4) モデルの学習

In [ ]:
# モデルのコンパイル
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# モデルの学習
history = model.fit(train_images, train_labels, epochs=10, validation_split = 0.2)

In [ ]:
# 学習過程の図示

# Access the training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Plot of the training and validation loss
epochs = np.arange(1, len(train_loss) + 1)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot　of the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## 5) モデルの検証

In [ ]:
# 検証データによる予測モデルの精度の検証
score = model.evaluate(test_images, test_labels, verbose = 0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

## 6) モデルの保存と復元

In [ ]:
# モデルの保存。HDF5ファイルで保存される。
model.save('/content/drive/MyDrive/Telakoya/tela5/model/cnn.h5')

In [ ]:
# モデルの復元。
from keras.models import load_model
model2 = load_model('/content/drive/MyDrive/Telakoya/tela5/model/cnn.h5')

In [ ]:
# 保存前のモデルによる予測結果
pred = model.predict(test_images[0:1])
pred

In [ ]:
# 復元したモデルによる予測結果
pred2 = model2.predict(test_images[0:1])
pred2

# 4. ChatGPTが提案したモデルによる分類

- 分類精度の高いモデルの設計を ChatGPT に作らせた場合。

- OpenAI が公開している ChatGPT を利用する場合、以下のリンクから登録すれば誰でも利用できる。
- GPT3.5から作られた ChatGPT は無利で使用できる。
- $20/monthの ChatGPt Plusに加入すると GPT-4 も利用できる。(2023年5月時点)

https://chat.openai.com/

In [ ]:
# Googe driveへのマウント。既に行われている場合は評価は不要。
from google.colab import drive
drive.mount('/content/drive')

## 1) ライブラリの読み込み

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt

## 2) データの読み込み

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

- モデルの損失関数がSparse Categorical Crossentropyの場合、正解データは一次元でよい。

## 3) モデルの設計

- 【ChatGPTに入力したprompt】
- I want to make a CNNN model using keras to classifiy cifar10 dataset with a high accuracy. Please sugget such a model.

In [ ]:
# ChatGPTが提案したモデル

import tensorflow as tf
from tensorflow.keras import layers

# Define the model
model = tf.keras.Sequential()

# Convolutional layer 1
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

# Convolutional layer 2
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

# Convolutional layer 3
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

# Flatten the output
model.add(layers.Flatten())

# Fully connected layers
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


## 4) モデルの学習

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=25, validation_split = 0.2)

In [ ]:
# モデルの学習過程

# Access the training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Plot of the training and validation loss
epochs = np.arange(1, len(train_loss) + 1)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot　of the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## 5) モデルの検証

In [ ]:
# 予測モデルの精度をTest dataで検証
score = model.evaluate(test_images, test_labels, verbose = 0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

- ChatGPTは、更なる予測精度の向上のため、以下のような suggestion も提案した。
- 1. Data Augmentation データの水増し (反転、回転、移動など)
- 2. より高度な ResNet, DenseNet, EfficientNet などの利用。
- 3. Hyperparameterのチューニング
- 4. 正則化 (regularization) L1, L2などで、次元削減を行う

## 6) モデルの保存と復元

In [ ]:
# モデルの保存。HDF5ファイルで保存される。
model.save('/content/drive/MyDrive/Telakoya/tela5/model/cnnC.h5')

In [ ]:
# モデルの復元。
from keras.models import load_model
model2 = load_model('/content/drive/MyDrive/Telakoya/tela5/model/cnnC.h5')

In [ ]:
# 保存前のモデルによる予測結果
pred = model.predict(test_images[0:1])
pred

In [ ]:
# 復元したモデルによる予測結果
pred2 = model2.predict(test_images[0:1])
pred2

# 5. Kaggleで提案されたモデルによる分類

- 世界的な機械学習のCompetitionであるKaggleにおいて、Cifar10の画像分類の銀メダリストのKedar氏が提案したモデル。

https://www.kaggle.com/code/kedarsai/cifar-10-88-accuracy-using-keras

https://www.kaggle.com/kedarsai

In [ ]:
# Googe driveへのマウント。既に行われている場合は評価は不要。
from google.colab import drive
drive.mount('/content/drive')

## 1) ライブラリの読み込み

In [ ]:
# 各種ライブラリの読み込み
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from keras.layers import BatchNormalization, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

## 2) データの読み込み

- モデルの損失関数が Cross Entropy のため、正解データは One Hot vectorに変換する

In [ ]:
# データの分割
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# データの値が0〜1になるように正規化。
train_images, test_images = train_images / 255.0, test_images / 255.0

# 正解データを One Hot vector に変換。
from keras.utils import to_categorical
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

## 3) モデルの設計

- ８層からなるCNN

https://www.kaggle.com/code/kedarsai/cifar-10-88-accuracy-using-keras

In [ ]:
# KagglerのKedar氏が公開したモデル
model6 = models.Sequential()
model6.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
model6.add(BatchNormalization())
model6.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPool2D((2, 2)))
model6.add(layers.Dropout(0.2))
model6.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPool2D((2, 2)))
model6.add(layers.Dropout(0.3))
model6.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPool2D((2, 2)))
model6.add(layers.Dropout(0.4))
model6.add(layers.Flatten())
model6.add(layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
model6.add(BatchNormalization())
model6.add(layers.Dropout(0.5))
model6.add(layers.Dense(10, activation='softmax'))

model6.summary()

## 4) モデルの学習

- モデルが大きく、学習を200回行うため、計算に1時間以上かかる可能性あり。

In [ ]:
# compile model
# opt = SGD(lr=0.001, momentum=0.9)
model6.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Image Data Generator , we are shifting image accross width and height also we are flipping the image horizantally.
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True,rotation_range=20)
it_train = datagen.flow(train_images, train_labels)
steps = int(train_images.shape[0] / 64)

# Be patient. It will take some time to train the due to 200 epochs of optimization.
history6=model6.fit_generator(it_train, epochs=200, steps_per_epoch=steps, validation_data=(test_images, test_labels))

In [ ]:
# モデルの学習過程

# Access the training history
train_loss = history6.history['loss']
val_loss = history6.history['val_loss']
train_accuracy = history6.history['accuracy']
val_accuracy = history6.history['val_accuracy']

# Plot of the training and validation loss
epochs = np.arange(1, len(train_loss) + 1)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot　of the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## 5) モデルの検証

In [ ]:
# 予測モデルの精度をtest dataで検証
score = model6.evaluate(test_images, test_labels, verbose = 0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

## 6) モデルの保存と復元

In [ ]:
# モデルの保存。HDF5ファイルで保存される。モデルの名称のcnnKは変更可能。
model6.save('/content/drive/MyDrive/Telakoya/tela5/model/cnnK.h5')

In [ ]:
# モデルの復元。
from keras.models import load_model
model62 = load_model('/content/drive/MyDrive/Telakoya/tela5/model/cnnK.h5')

In [ ]:
# 保存前のモデルによる予測結果
pred = model6.predict(test_images[0:1])
pred

In [ ]:
# 復元したモデルによる予測結果
pred2 = model62.predict(test_images[0:1])
pred2